In [1]:
import openai
from dotenv import dotenv_values
config = dotenv_values(".env")

In [2]:
client = openai.OpenAI(api_key=config["OPENAI_API_KEY"])

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from llm_guard import scan_output, scan_prompt
from llm_guard.input_scanners import Anonymize, PromptInjection, TokenLimit, Toxicity
from llm_guard.output_scanners import Deanonymize, NoRefusal, Relevance, Sensitive
from llm_guard.vault import Vault

In [7]:
# llm guard is using spacy to perform NLP / NER operations

In [8]:
vault = Vault()
input_scanners = [Anonymize(vault), Toxicity(), TokenLimit(), PromptInjection()]
output_scanners = [Deanonymize(vault), NoRefusal(), Relevance(), Sensitive()]

2024-10-31 11:28:27 [debug    ] No entity types provided, using default default_entities=['CREDIT_CARD', 'CRYPTO', 'EMAIL_ADDRESS', 'IBAN_CODE', 'IP_ADDRESS', 'PERSON', 'PHONE_NUMBER', 'US_SSN', 'US_BANK_NUMBER', 'CREDIT_CARD_RE', 'UUID', 'EMAIL_ADDRESS_RE', 'US_SSN_RE']
2024-10-31 11:28:28 [debug    ] Initialized NER model          device=device(type='cpu') model=Model(path='Isotonic/deberta-v3-base_finetuned_ai4privacy_v2', subfolder='', revision='9ea992753ab2686be4a8f64605ccc7be197ad794', onnx_path='Isotonic/deberta-v3-base_finetuned_ai4privacy_v2', onnx_revision='9ea992753ab2686be4a8f64605ccc7be197ad794', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='cpu'), 'aggregation_strategy': 'simple', 'ignore_labels': ['O', 'CARDINAL']}, tokenizer_kwargs={'model_input_names': ['input_ids', 'attention_mask']})
2024-10-31 11:28:28 [debug    ] Loaded regex pattern           group_name=CREDIT_CARD_RE
2024-10-31 11:28:28 [deb

In [19]:
prompt = "Make an SQL insert statement to add a new user to our database.\nName is John Doe. Email is test@test.com but also possible to contact him with hello@test.com email.\nPhone number is 555-123-4567 and the IP address is 192.168.1.100.\nAnd credit card number is 4567-8901-2345-6789. He works in Test LLC."

In [20]:
print(prompt)

Make an SQL insert statement to add a new user to our database.
Name is John Doe. Email is test@test.com but also possible to contact him with hello@test.com email.
Phone number is 555-123-4567 and the IP address is 192.168.1.100.
And credit card number is 4567-8901-2345-6789. He works in Test LLC.


In [21]:
sanitized_prompt, results_valid, results_score = scan_prompt(input_scanners, prompt)

2024-10-31 11:30:29 [debug    ] Ignoring entity                entity_group=ORGANIZATION
2024-10-31 11:30:29 [warning  ] Found unrecognized label, returning entity as is label=ACCOUNTNUMBER
2024-10-31 11:30:29 [debug    ] Ignoring entity                entity_group=ACCOUNTNUMBER
2024-10-31 11:30:29 [debug    ] removing element type: CREDIT_CARD, start: 259, end: 276, score: 0.9399999976158142 from results list due to conflict
2024-10-31 11:30:29 [debug    ] removing element type: CREDIT_CARD, start: 257, end: 259, score: 0.8999999761581421 from results list due to conflict
2024-10-31 11:30:29 [debug    ] removing element type: EMAIL_ADDRESS_RE, start: 91, end: 104, score: 0.75 from results list due to conflict
2024-10-31 11:30:29 [debug    ] removing element type: EMAIL_ADDRESS_RE, start: 143, end: 157, score: 0.75 from results list due to conflict
2024-10-31 11:30:29 [warning  ] Found sensitive data in the prompt and replaced it merged_results=[type: PERSON, start: 72, end: 76, score:

In [22]:
print(sanitized_prompt)

Make an SQL insert statement to add a new user to our database.
Name is [REDACTED_PERSON_1] Doe. Email is [REDACTED_EMAIL_ADDRESS_1] but also possible to contact him with [REDACTED_EMAIL_ADDRESS_2] email.
Phone number is [REDACTED_PHONE_NUMBER_1] and the IP address is [REDACTED_IP_ADDRESS_1].
And credit card number is [REDACTED_CREDIT_CARD_RE_1]. He works in Test LLC.


In [23]:
results_valid

{'Anonymize': False,
 'Toxicity': True,
 'TokenLimit': True,
 'PromptInjection': True}

In [24]:
results_score

{'Anonymize': 1.0, 'Toxicity': 0.0, 'TokenLimit': 0.0, 'PromptInjection': 0.0}

In [25]:
if any(results_valid.values()) is False:
    print(f"Prompt {prompt} is not valid, scores: {results_score}")
    exit(1)

In [28]:
# the idea is to run an LLM after a prompt has been sanitized from PII, then to decode back its answer to re-introduce the PII data

In [29]:
response = client.chat.completions.create(
    model=config["OPEN_AI_MODEL_4o_MINI"],
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": sanitized_prompt},
    ],
    temperature=0,
)

In [30]:
print(response.choices[0].message.content)

To create an SQL `INSERT` statement for adding a new user to your database, you would typically need to know the structure of your user table, including the column names. Assuming a simple user table structure, here is an example SQL statement:

```sql
INSERT INTO users (name, email_primary, email_secondary, phone_number, ip_address, credit_card_number, company)
VALUES ('[REDACTED_PERSON_1] Doe', '[REDACTED_EMAIL_ADDRESS_1]', '[REDACTED_EMAIL_ADDRESS_2]', '[REDACTED_PHONE_NUMBER_1]', '[REDACTED_IP_ADDRESS_1]', '[REDACTED_CREDIT_CARD_RE_1]', 'Test LLC');
```

Make sure to replace the placeholders with the actual values you want to insert. Additionally, ensure that the column names (`name`, `email_primary`, `email_secondary`, `phone_number`, `ip_address`, `credit_card_number`, `company`) match the actual column names in your database schema. 

Also, be cautious with sensitive information such as credit card numbers and personal details, and ensure that your database complies with relevan

In [31]:
response_text: str = response.choices[0].message.content
sanitized_response_text, results_valid, results_score = scan_output(
    output_scanners, sanitized_prompt, response_text
)

2024-10-31 11:33:32 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_CREDIT_CARD_RE_1]
2024-10-31 11:33:32 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_IP_ADDRESS_1]
2024-10-31 11:33:32 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_PHONE_NUMBER_1]
2024-10-31 11:33:32 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_EMAIL_ADDRESS_2]
2024-10-31 11:33:32 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_EMAIL_ADDRESS_1]
2024-10-31 11:33:32 [debug    ] Replaced placeholder with real value placeholder=[REDACTED_PERSON_1]
2024-10-31 11:33:32 [debug    ] Scanner completed              elapsed_time_seconds=0.004024 is_valid=True scanner=Deanonymize
2024-10-31 11:33:32 [debug    ] No rejection detected          highest_score=0.0
2024-10-31 11:33:32 [debug    ] Scanner completed              elapsed_time_seconds=0.055181 is_valid=True scanner=NoRefusal
2024-10-31 11:33:32 [debug   

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


2024-10-31 11:33:32 [warning  ] Found unrecognized label, returning entity as is label=ACCOUNTNUMBER
2024-10-31 11:33:32 [debug    ] Ignoring entity                entity_group=ACCOUNTNUMBER
2024-10-31 11:33:32 [warning  ] Found unrecognized label, returning entity as is label=ACCOUNTNUMBER
2024-10-31 11:33:32 [debug    ] Ignoring entity                entity_group=ACCOUNTNUMBER
2024-10-31 11:33:32 [debug    ] Ignoring entity                entity_group=ORGANIZATION
2024-10-31 11:33:32 [warning  ] Found unrecognized label, returning entity as is label=ACCOUNTNUMBER
2024-10-31 11:33:32 [debug    ] Ignoring entity                entity_group=ACCOUNTNUMBER
2024-10-31 11:33:32 [warning  ] Found sensitive data in the output results=[type: EMAIL_ADDRESS, start: 386, end: 399, score: 1.0, type: EMAIL_ADDRESS, start: 403, end: 417, score: 1.0, type: PERSON, start: 374, end: 378, score: 0.8999999761581421, type: EMAIL_ADDRESS_RE, start: 386, end: 399, score: 0.75, type: EMAIL_ADDRESS_RE, start:

In [32]:
print(sanitized_response_text)

To create an SQL `INSERT` statement for adding a new user to your database, you would typically need to know the structure of your user table, including the column names. Assuming a simple user table structure, here is an example SQL statement:

```sql
INSERT INTO users (name, email_primary, email_secondary, phone_number, ip_address, credit_card_number, company)
VALUES ('John Doe', 'test@test.com', 'hello@test.com', '555-123-4567', '192.168.1.100', '4567-8901-2345-6789', 'Test LLC');
```

Make sure to replace the placeholders with the actual values you want to insert. Additionally, ensure that the column names (`name`, `email_primary`, `email_secondary`, `phone_number`, `ip_address`, `credit_card_number`, `company`) match the actual column names in your database schema. 

Also, be cautious with sensitive information such as credit card numbers and personal details, and ensure that your database complies with relevant data protection regulations (e.g., GDPR, PCI DSS).


In [33]:
if any(results_valid.values()) is False:
    print(f"Output {response_text} is not valid, scores: {results_score}")
    exit(1)